In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Load datasets
gate_samples_df = pd.read_csv("final_2500_gate_samples.csv")
transistor_data_df = pd.read_csv("/content/synthetic_cmos_power_dataset (2) (1) (1) (2) (1) (1).csv")
counter_shift_register_df = pd.read_csv("modified_shift_register_data.csv")

# Model training for combinational logic circuits
X = gate_samples_df[['Circuit_Type', 'Bit_Width']]
y = gate_samples_df[['AND_Gates', 'OR_Gates', 'NOT_Gates', 'XOR_Gates']]

preprocessor = ColumnTransformer([
    ('cat', OneHotEncoder(handle_unknown='ignore'), ['Circuit_Type'])
], remainder='passthrough')

model = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model.fit(X_train, y_train)

# Transistor and power metrics
avg_data = transistor_data_df.groupby('Circuit_Type').agg({
    'No_Of_Transistors': 'mean',
    'Static_Power': 'mean',
    'Dynamic_Power': 'mean',
    'Short_ckt_Power': 'mean'
}).to_dict()

defaults = {
    'AND': {'No_Of_Transistors': 4, 'Static_Power': 1e-9, 'Dynamic_Power': 1e-6, 'Short_ckt_Power': 1e-9},
    'OR': {'No_Of_Transistors': 4, 'Static_Power': 1e-9, 'Dynamic_Power': 1e-6, 'Short_ckt_Power': 1e-9},
    'XOR': {'No_Of_Transistors': 8, 'Static_Power': 2e-9, 'Dynamic_Power': 2e-6, 'Short_ckt_Power': 2e-9},
    'NOT': {'No_Of_Transistors': 2, 'Static_Power': 0.5e-9, 'Dynamic_Power': 0.5e-6, 'Short_ckt_Power': 0.5e-9},
    'DFF': {'No_Of_Transistors': 18, 'Static_Power': 0.7e-9, 'Dynamic_Power': 0.7e-6, 'Short_ckt_Power': 0.7e-9},
}
gate_metrics = {gate: {
    'No_Of_Transistors': avg_data['No_Of_Transistors'].get(gate, defaults[gate]['No_Of_Transistors']),
    'Static_Power': avg_data['Static_Power'].get(gate, defaults[gate]['Static_Power']),
    'Dynamic_Power': avg_data['Dynamic_Power'].get(gate, defaults[gate]['Dynamic_Power']),
    'Short_ckt_Power': avg_data['Short_ckt_Power'].get(gate, defaults[gate]['Short_ckt_Power']),
} for gate in defaults}

# Flip-flop lookup
ff_lookup = counter_shift_register_df.set_index(['Circuit_Type', 'Bit_Width'])['Flip_Flops'].to_dict()

# Prediction and power estimation
def predict_transistors_and_power_user_input():
    circuit_type = input(f"Enter circuit type: ").strip()

    try:
        bit_width = int(input("Enter bit width: "))
        if bit_width < 1 or bit_width > 32:
            raise ValueError
    except ValueError:
        print("Invalid bit width.")
        return

    plot_choice = input("Do you want to plot Activity Factor vs Total Power? (yes/no): ").strip().lower()
    plot_graph = plot_choice == 'yes'

    # Circuit logic
    if circuit_type in ['Ripple_Adder', 'Ripple_Subtractor', 'Half_Adder']:
        sample_input = pd.DataFrame([{'Circuit_Type': circuit_type, 'Bit_Width': bit_width}])
        predicted_gates = model.predict(sample_input).astype(int)[0]
        gate_types = ['AND', 'OR', 'NOT', 'XOR']
        gate_counts = dict(zip(gate_types, predicted_gates))
        print("\nUsing ML model prediction for combinational circuit.")
    elif circuit_type in ['Counter', 'Shift_Register']:
        key = (circuit_type, bit_width)
        flip_flops = ff_lookup.get(key)
        if flip_flops is None:
            print("Bit width not found in dataset.")
            return
        gate_counts = {'DFF': flip_flops}
        print("\nUsing flip-flop count for sequential circuit.")

    # Power calculation
    total_transistors = 0
    total_static_power = 0
    total_dynamic_power = 0
    total_short_power = 0

    print("\nGate/Flip-Flop Counts and Power Breakdown:\n")
    for gate, count in gate_counts.items():
        metrics = gate_metrics[gate]
        transistors = count * metrics['No_Of_Transistors']
        static_power = count * metrics['Static_Power']
        dynamic_power = count * metrics['Dynamic_Power']
        short_power = count * metrics['Short_ckt_Power']

        total_transistors += transistors
        total_static_power += static_power
        total_dynamic_power += dynamic_power
        total_short_power += short_power

        print(f"{gate} Gates: {count} × {metrics['No_Of_Transistors']} = {transistors} transistors")
        print(f"            Static Power: {static_power:.2e} W")
        print(f"            Dynamic Power: {dynamic_power:.2e} W")
        print(f"            Short-Circuit Power: {short_power:.2e} W\n")

    total_power = total_static_power + total_dynamic_power + total_short_power
    print(f"Total Transistors: {int(total_transistors)}")
    print(f"Total Static Power: {total_static_power:.2e} W")
    print(f"Total Dynamic Power: {total_dynamic_power:.2e} W")
    print(f"Total Short-Circuit Power: {total_short_power:.2e} W")
    print(f"Total Power Consumption: {total_power:.2e} W")

    # Optional plot
    if plot_graph:
        activity_factors = np.linspace(0.1, 1.0, 10)
        results = []

        for alpha in activity_factors:
            dynamic_power = sum(gate_counts[g] * gate_metrics[g]['Dynamic_Power'] * alpha for g in gate_counts)
            static_power = sum(gate_counts[g] * gate_metrics[g]['Static_Power'] for g in gate_counts)
            short_power = sum(gate_counts[g] * gate_metrics[g]['Short_ckt_Power'] for g in gate_counts)
            total = static_power + dynamic_power + short_power
            results.append({
                'Activity_Factor': alpha,
                'Static_Power': static_power,
                'Dynamic_Power': dynamic_power,
                'Short_ckt_Power': short_power,
                'Total_Power': total
            })

        df_results = pd.DataFrame(results)
        plt.figure(figsize=(8, 6))
        plt.plot(df_results['Activity_Factor'], df_results['Total_Power'], marker='o', color='blue')
        plt.title(f'Activity Factor vs Total Power for {bit_width}-bit {circuit_type}')
        plt.xlabel('Activity Factor (α)')
        plt.ylabel('Total Power (W)')
        plt.grid(True)
        plt.tight_layout()
        plt.show()

        filename = f"{bit_width}bit_{circuit_type.lower()}_activity_power.csv".replace(" ", "_")
        df_results.to_csv(filename, index=False)
        print(f"\nCSV saved to: {filename}")


predict_transistors_and_power_user_input()



Enter circuit type: Shift_Register
Enter bit width: 4
Do you want to plot Activity Factor vs Total Power? (yes/no): no

Using flip-flop count for sequential circuit.

Gate/Flip-Flop Counts and Power Breakdown:

DFF Gates: 4 × 18 = 72 transistors
            Static Power: 2.80e-09 W
            Dynamic Power: 2.80e-06 W
            Short-Circuit Power: 2.80e-09 W

Total Transistors: 72
Total Static Power: 2.80e-09 W
Total Dynamic Power: 2.80e-06 W
Total Short-Circuit Power: 2.80e-09 W
Total Power Consumption: 2.81e-06 W
